In [4]:
import csv
import numpy as np

import scipy
import scipy.sparse

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.cross_validation import train_test_split

from sklearn.metrics import accuracy_score

In [2]:
def get_data():
    train_file = csv.reader(open('train.data'), delimiter='\t')

    train_set = [x[1:] for x in train_file]
    test_file = csv.reader(open('test.data'), delimiter='\t')

    test_data = [x[1] for x in test_file]
    test_data = [x.replace('\\n', ' ') for x in test_data[1:]]

    train_data = [x[1].replace('\\n', ' ') for x in train_set[1:]]
    train_target = np.array([int(x[0]) for x in train_set[1:]])

    return train_data, train_target, test_data

In [5]:
train_data, train_target, test_data = get_data()

x_train, x_validate, y_train, y_validate = train_test_split(train_data, train_target, test_size=0.5, random_state=42)

In [6]:
def tfidf_features(x_train, x_test):
    d = TfidfVectorizer(ngram_range=(1, 3), smooth_idf=False)
    train_features = d.fit_transform(x_train)
    test_features = d.transform(x_test)

    d = TfidfVectorizer(analyzer='char', ngram_range=(1, 4), smooth_idf=False)
    train_chars = d.fit_transform(x_train)
    test_chars = d.transform(x_test)

    del d

    train_length = np.array([len(text) for text in x_train])
    test_length = np.array([len(text) for text in x_test])

    train_length = scipy.sparse.csr_matrix(train_length).T
    test_length = scipy.sparse.csr_matrix(test_length).T

    train_features = scipy.sparse.hstack([train_features, train_chars, train_length])
    test_features = scipy.sparse.hstack([test_features, test_chars, test_length])

    return train_features, test_features

In [7]:
def get_classificator(x_train, y_train):
    clf = LogisticRegression(penalty='l1', max_iter=500000, C=3)
    clf.fit(x_train, y_train)

    return clf

def classify(x_train, y_train, x_test):
    train_features, test_features = tfidf_features(x_train, x_test)

    clf = get_classificator(train_features, y_train)

    return clf.predict_proba(test_features)

def get_ans(probs):
    ans = probs.argmax(axis=1) + 1
    return ans

In [8]:
probs = classify(x_train, y_train, x_validate)
p_validate = get_ans(probs)

print(accuracy_score(y_validate, p_validate))

0.599098923389


Итого получили точность чуть меньше 60%. Учитывая, что сравнивали точное поподание оценок, а не как-то более лояльно это хороший результат для анализа тональностей.